For a first step, I will re-write the code from section 1.4.4 --- the C program for the single-server queueing system --- to be more extensible. Currently it relies extremely heavily on global variables and is difficult to debug.

The following is a rough implementation of the provided C program in Python.

In [53]:
import numpy as np
import sys

MEAN_INTERARRIVAL = 1.0
MEAN_SERVICE = 0.5

NUM_DELAYS_REQUIRED = 1000
NUM_EVENTS = 2
SIM_TIME = 0.0
Q_LIMIT = 1000

def main() -> int:
    """Driver for the simulation."""
    global num_custs_delayed, next_event_type

    # initialize the simulation
    initialize()

    while num_custs_delayed < NUM_DELAYS_REQUIRED:
        # determine the next event, advance sim clock
        timing()

        # update time-avg statistical accumulators
        update_time_avg_stats()

        # invoke the appropriate event function
        if next_event_type == 0:
            arrive()
        elif next_event_type == 1:
            depart()

    # print report
    report()


def initialize():
    print("initializing")
    global SIM_TIME, server_status, num_in_q, time_last_event, num_custs_delayed, total_of_delays, area_num_in_q, area_server_status, time_next_event
    # init the sim clock
    SIM_TIME = 0.0

    # init the state variables
    server_status = 'IDLE'
    num_in_q = 0
    time_last_event = 0.0

    # init the statistical counters
    global time_arrival
    time_arrival = []
    num_custs_delayed = 0
    total_of_delays = 0.0
    area_num_in_q = 0.0
    area_server_status = 0.0

    # init the event list
    time_next_event = [
        SIM_TIME + expon(MEAN_INTERARRIVAL),
        np.inf
    ]

def expon(mean: float) -> float:
    # return np.random.exponential(scale=1/mean)
    return np.random.exponential(scale=mean)


def timing() -> None:
    """Determine next event type, and advance the simulation clock."""
    # print("timing")
    global time_next_event, next_event_type, SIM_TIME
    min_time_next_event = np.inf

    next_event_type = None
    # determine the type of the next event to occur
    for i in range(0, NUM_EVENTS):
        if time_next_event[i] < min_time_next_event:
            min_time_next_event = time_next_event[i]
            next_event_type = i
    # if next_event_type == 0:
    #     print("next event is an arrival")
    # elif next_event_type == 1:
    #     print("next event is a departure")
    # check if event list is empty
    if next_event_type is None:
        print(f"\nEvent list is empty at time {SIM_TIME:f}")
        sys.exit(1)

    # advance the sim clock
    SIM_TIME = min_time_next_event


def update_time_avg_stats():
    global SIM_TIME, time_last_event
    time_since_last_event = SIM_TIME - time_last_event
    time_last_event = SIM_TIME

    # update area under number-in-queue function
    global area_num_in_q, num_in_q
    area_num_in_q += num_in_q * time_since_last_event

    # update area under server-busy indicator function
    global area_server_status, server_status
    server_status_indic = 1 if server_status == 'BUSY' else 0
    area_server_status += server_status_indic * time_since_last_event

def arrive():
    # print("arriving")
    global time_next_event, SIM_TIME

    # schedule the next arrival event
    time_next_event[0] = SIM_TIME + expon(MEAN_INTERARRIVAL)

    # check if server is busy
    global server_status
    if server_status == 'BUSY':
        # increment number of customers in queue
        global num_in_q
        num_in_q += 1

        # check to see if queue overflowed
        if num_in_q > Q_LIMIT:
            print(f"\nOverflow of the queue time_arrival at time {SIM_TIME:f}.")
            sys.exit(2)

        # add customer to queue, recording arrival time
        global time_arrival
        time_arrival.append(SIM_TIME)

    # else, if server is idle
    elif server_status == 'IDLE':
        # arriving customer has a delay of 0
        global total_of_delays
        delay = 0.0
        total_of_delays += delay

        global num_custs_delayed
        num_custs_delayed += 1
        server_status = 'BUSY'

        # schedule a departure event
        time_next_event[1] = SIM_TIME + expon(MEAN_SERVICE)

def depart():
    # print("departing")
    global num_in_q, time_next_event
    if num_in_q == 0:
        # queue is empty, so server becomes idle
        global server_status
        server_status = 'IDLE'
        time_next_event[1] = np.inf

    else:
        # queue is not empty, so someone waiting will start service
        num_in_q -= 1

        global SIM_TIME, time_arrival, total_of_delays
        delay = SIM_TIME - time_arrival[0]
        total_of_delays += delay

        global num_custs_delayed
        num_custs_delayed += 1

        # schedule next departure
        time_next_event[1] = SIM_TIME + expon(MEAN_SERVICE)

        # shift queue
        time_arrival = time_arrival[1:]

def report() -> None:
    global total_of_delays, num_custs_delayed

    print(f"\n\nAverage delay in queue {total_of_delays/num_custs_delayed:11.3f} minutes\n\n")

    global area_num_in_q, SIM_TIME
    print(f"Average number in queue {area_num_in_q/SIM_TIME:10.3f}\n\n")

    global area_server_status
    print(f"Server utilization {area_server_status / SIM_TIME:15.3f}\n\n")

    print(f"Time simulation ended {SIM_TIME:12.3f} minutes")

In [56]:
main()

initializing


Average delay in queue       0.425 minutes


Average number in queue      0.420


Server utilization           0.497


Time simulation ended     1011.774 minutes


I will try to rewrite this in a more sensible way via objects and returning values instead of using global variables, when possible.

In [138]:
import numpy as np
import sys

from dataclasses import dataclass, field

@dataclass
class State:
    server_status: str = 'IDLE'
    num_in_q: int = 0
    time_last_event: float = 0.0
    time_arrival: list[float] = field(default_factory=list)

@dataclass
class StatAccumulator:
    num_custs_delayed: int = 0
    total_of_delays: float = 0.0
    area_num_in_q: float = 0.0
    num_in_q: int = 0
    area_server_status: float = 0.0

    def update_time_avg_stats(self, state: State) -> None:
        global SIM_TIME
        time_since_last_event = SIM_TIME - state.time_last_event
        state.time_last_event = SIM_TIME

        # update area under number-in-queue function
        self.num_in_q = state.num_in_q
        self.area_num_in_q += self.num_in_q * time_since_last_event

        # update area under server-busy indicator function
        server_status_indic = 1 if state.server_status == 'BUSY' else 0
        self.area_server_status += server_status_indic * time_since_last_event

NUM_EVENTS = 2

@dataclass
class EventList:
    time_next_event: list[float] = field(default_factory=list)

    @property
    def next_event_type(self) -> int:
        type_ = None
        min_time_next_event = np.inf
        for i in range(0, NUM_EVENTS):
            if self.time_next_event[i] < min_time_next_event:
                min_time_next_event = self.time_next_event[i]
                type_ = i
        return type_


MEAN_INTERARRIVAL = 1.0
MEAN_SERVICE = 0.5

NUM_DELAYS_REQUIRED = 1000

SIM_TIME = 0.0
Q_LIMIT = 2000


def main() -> int:
    """Driver for the simulation."""

    print("Single-server queuing system\n")
    print(f"Mean interarrival time {MEAN_INTERARRIVAL:11.3f} minutes\n")
    print(f"Mean service time {MEAN_SERVICE:11.3f} minutes\n")
    print(f"Number of customers {NUM_DELAYS_REQUIRED:14d}\n")

    # initialize the simulation
    state, stat_accum, event_list = initialize()

    while stat_accum.num_custs_delayed < NUM_DELAYS_REQUIRED:
        # determine the next event, advance sim clock
        timing(event_list)

        # update time-avg statistical accumulators
        stat_accum.update_time_avg_stats(state)
        # print(state.time_arrival)

        # invoke the appropriate event function
        if event_list.next_event_type == 0:
            arrive(state, stat_accum, event_list)
        elif event_list.next_event_type == 1:
            depart(state, stat_accum, event_list)

    # print report
    report(stat_accum)

    return state, stat_accum, event_list

def initialize() -> tuple[State, StatAccumulator, EventList]:
    global SIM_TIME
    # init the sim clock
    SIM_TIME = 0.0

    # init the state variables
    state = State()

    # init the statistical counters
    stat_accum = StatAccumulator()

    # init the event list
    event_list = EventList([
        SIM_TIME + expon(MEAN_INTERARRIVAL),
        np.inf
    ])

    return state, stat_accum, event_list

def expon(mean: float) -> float:
    # return np.random.exponential(scale=1/mean)
    return np.random.exponential(scale=mean)


def timing(event_list: EventList) -> None:
    """Determine next event type, and advance the simulation clock."""
    # print("timing")
    global SIM_TIME
    next_event_type = event_list.next_event_type
    
    # if next_event_type == 0:
    #     print("next event will be an arrival")
    # elif next_event_type == 1:
    #     print("next event will be a departure")
    if next_event_type is None:
        print(f"\nEvent list is empty at time {SIM_TIME:f}")
        sys.exit(1)

    # advance the sim clock
    SIM_TIME = event_list.time_next_event[next_event_type]


def arrive(state: State, stat_accum: StatAccumulator, event_list: EventList):
    # print("Arriving")
    global SIM_TIME

    # schedule the next arrival event
    event_list.time_next_event[0] = SIM_TIME + expon(MEAN_INTERARRIVAL)

    # check if server is busy
    if state.server_status == 'BUSY':
        # print("   going to queue")
        # increment number of customers in queue
        state.num_in_q += 1

        # check to see if queue overflowed
        if state.num_in_q > Q_LIMIT:
            print(f"\nOverflow of the queue time_arrival at time {SIM_TIME:f}.")
            sys.exit(2)

        # add customer to queue, recording arrival time
        state.time_arrival.append(SIM_TIME)

    # else, if server is idle
    elif state.server_status == 'IDLE':
        # arriving customer has a delay of 0
        delay = 0.0
        stat_accum.total_of_delays += delay

        stat_accum.num_custs_delayed += 1
        state.server_status = 'BUSY'

        # schedule |a departure event
        event_list.time_next_event[1] = SIM_TIME + expon(MEAN_SERVICE)

def depart(state: State, stat_accum: StatAccumulator, event_list: EventList):
    # print("departing")
    if state.num_in_q == 0:
        # queue is empty, so server becomes idle
        state.server_status = 'IDLE'
        event_list.time_next_event[1] = np.inf

    else:
        # queue is not empty, so someone waiting will start service
        state.num_in_q -= 1
        
        global SIM_TIME
        delay = SIM_TIME - state.time_arrival[0]
        stat_accum.total_of_delays += delay

        stat_accum.num_custs_delayed += 1

        # schedule next departure
        event_list.time_next_event[1] = SIM_TIME + expon(MEAN_SERVICE)

        # shift queue
        state.time_arrival = state.time_arrival[1:]

def report(stat_accum: StatAccumulator) -> None:
    """Print results of the simulation run."""
    print(f"\nAverage delay in queue {stat_accum.total_of_delays/stat_accum.num_custs_delayed:11.3f} minutes\n")

    global SIM_TIME
    print(f"Average number in queue {stat_accum.area_num_in_q/SIM_TIME:10.3f}\n")

    print(f"Server utilization {stat_accum.area_server_status / SIM_TIME:15.3f}\n")

    print(f"Time simulation ended {SIM_TIME:12.3f} minutes")

In [139]:
state, stat_accum, event_list = main()

Single-server queuing system

Mean interarrival time       1.000 minutes

Mean service time       0.500 minutes

Number of customers           1000


Average delay in queue       0.540 minutes

Average number in queue      0.566

Server utilization           0.533

Time simulation ended      953.731 minutes
